<a href="https://colab.research.google.com/github/judem-21/Natural-Language-Processing/blob/main/Sentiment%20Analysis/Sentiment_Analysis_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SA Model

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [ ]:
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.optim.lr_scheduler import StepLR

In [ ]:
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset['train'][9]

{'id': 793, 'text': '  sad face.', 'label': 0, 'sentiment': 'negative'}

In [ ]:
def filter_none(example):
    return example["text"] is not None

dataset['test'] = dataset['test'].filter(filter_none)
dataset['validation']=dataset['validation'].filter(filter_none)
dataset['train']=dataset['train'].filter(filter_none)

Filter:   0%|          | 0/5206 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5205 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31232 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
    tokenised = tokenizer(text=batch["text"], truncation=True)
    tokenised["labels"] = batch["label"]
    return tokenised

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'text', 'sentiment'])

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator
)

In [ ]:
for batch in train_loader:
    print({k: v.shape for k, v in batch.items()})  # Shows input_ids, attention_mask shapes
    print(f'Input Ids:{batch["input_ids"]}')
    print(f'Attention Mask:{batch["attention_mask"]}')
    print(f'Labels:{batch["labels"]}')
    break

{'input_ids': torch.Size([32, 119]), 'token_type_ids': torch.Size([32, 119]), 'attention_mask': torch.Size([32, 119]), 'labels': torch.Size([32])}
Input Ids:tensor([[  101,  2748,  3531,  ...,     0,     0,     0],
        [  101,  2919,  2154,  ...,     0,     0,     0],
        [  101,  1045,  2097,  ...,     0,     0,     0],
        ...,
        [  101,  2074,  2893,  ...,     0,     0,     0],
        [  101, 10166,  1010,  ...,     0,     0,     0],
        [  101,  2123,  1036,  ...,     0,     0,     0]])
Attention Mask:tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Labels:tensor([1, 1, 1, 2, 1, 2, 2, 0, 2, 0, 0, 2, 1, 1, 0, 1, 2, 1, 1, 0, 2, 0, 0, 2,
        0, 0, 2, 2, 2, 2, 2, 0])


In [ ]:
class SentimentAnalyser(nn.Module):
  def __init__(self,embedding_dim,vocab_size,num_rec_layers,dropout,pad_idx,bidirectional=True):
    super().__init__()
    #self.embedding_dim=embedding_dim
    self.embeddings=nn.Embedding(vocab_size,embedding_dim,padding_idx=pad_idx)
    self.linear=nn.Linear(embedding_dim,3)
    self.lstm=nn.LSTM(input_size=embedding_dim,num_layers=num_rec_layers,hidden_size=embedding_dim,bidirectional=bidirectional,batch_first=True,dropout=dropout)
    self.dropout=nn.Dropout(dropout)
    self.relu=nn.ReLU()

  def forward(self,input):
    input_embeddings=self.embeddings(input)
    _,(out,_)=self.lstm(input_embeddings)
    #print(f'Out:{out.shape},and permute shape:{torch.mean(out.permute(1,0,2),dim=1).shape}')
    out=self.linear(self.dropout(torch.mean(out.permute(1,0,2),dim=1)))
    return self.relu(out)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path='/content/drive/MyDrive/SentimentAnalysis/checkpoint.pth'
legend={0:'Negative',1:'Neutral',2:'Positive'}
pad_idx = tokenizer.pad_token_id
model=SentimentAnalyser(embedding_dim=512,vocab_size=tokenizer.vocab_size,num_rec_layers=2,
dropout=0.1,pad_idx=pad_idx, bidirectional=True).to(device)
checkpoint=torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
softmax=nn.Softmax(dim=1)

In [ ]:
num_epochs,lr=100,0.001
# Learning rate scheduler: reduce LR by gamma every 'step_size' epochs
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # halve LR every 5 epochs
loss_fn=nn.CrossEntropyLoss()
min_loss=checkpoint['min_loss']
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
def accuracy(outputs,labels,training=False):
  if not training: return (outputs==labels).sum()/len(labels)
  return (softmax(outputs).argmax(dim=1)==labels).sum()/len(labels)

In [ ]:
epoch_losses=[]
model.train()
for epoch in range(1,num_epochs+1):
  batch_losses=[]
  for idx,batch in enumerate(train_loader):
    optimizer.zero_grad()
    inputs=batch['input_ids'].to(device)
    labels=batch['labels'].to(device)
    logits=model(inputs)
    loss=loss_fn(logits,labels)
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())
    if (epoch==1 or epoch%10==0) and (idx+1)%100==0:
      print(f'Epoch {epoch}, Batch {idx+1}, Batch Loss:{batch_losses[-1]:.8f} and accuracy:{accuracy(logits,labels,training=True)*100}%')
  epoch_losses.append(np.mean(batch_losses))
  scheduler.step()
  if epoch==1 or epoch%10==0:
    print(f'Epoch {epoch}, Learning Rate={scheduler.get_last_lr()[0]}')
  print(f'Epoch {epoch}, Epoch Loss:{epoch_losses[-1]:.4f}')
  if epoch_losses[-1]<min_loss:
    min_loss=epoch_losses[-1]
    checkpoint={'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict(),'min_loss':epoch_losses[-1].item()}
    torch.save(checkpoint,checkpoint_path)
    print('Model saved!')
  print()

Epoch 1, Batch 100, Batch Loss:0.00234770 and accuracy:100.0%
Epoch 1, Batch 200, Batch Loss:0.00174750 and accuracy:100.0%
Epoch 1, Batch 300, Batch Loss:0.01880415 and accuracy:100.0%
Epoch 1, Batch 400, Batch Loss:0.00063757 and accuracy:100.0%
Epoch 1, Batch 500, Batch Loss:0.00139569 and accuracy:100.0%
Epoch 1, Batch 600, Batch Loss:0.04056843 and accuracy:96.875%
Epoch 1, Batch 700, Batch Loss:0.00707080 and accuracy:100.0%
Epoch 1, Batch 800, Batch Loss:0.00831299 and accuracy:100.0%
Epoch 1, Batch 900, Batch Loss:0.02338232 and accuracy:100.0%
Epoch 1, Learning Rate=0.001
Epoch 1, Epoch Loss:0.0242



KeyboardInterrupt: 

In [ ]:
'''checkpoint={'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict(),'min_loss':epoch_losses[-1].item()}
torch.save(checkpoint,'/content/drive/MyDrive/SentimentAnalysis/checkpoint.pth')'''
#

In [ ]:
test_loader=DataLoader(tokenized_dataset['test'],batch_size=32,collate_fn=data_collator)

In [ ]:
model.eval()
for idx,batch in enumerate(test_loader):
  if (idx+1)%10!=0:continue
  with torch.no_grad():
    logits=model(batch['input_ids'].to(device))
    logits=softmax(logits).argmax(dim=1)
    #print(f'Logits shape: {logits.shape}')
    print(f"Batch: {idx+1}, Accuracy: {accuracy(logits,batch['labels'].to(device))*100}%\n")

Batch: 10, Accuracy: 65.625%

Batch: 20, Accuracy: 62.5%

Batch: 30, Accuracy: 68.75%

Batch: 40, Accuracy: 71.875%

Batch: 50, Accuracy: 59.375%

Batch: 60, Accuracy: 65.625%

Batch: 70, Accuracy: 65.625%

Batch: 80, Accuracy: 71.875%

Batch: 90, Accuracy: 59.375%

Batch: 100, Accuracy: 62.5%

Batch: 110, Accuracy: 65.625%

Batch: 120, Accuracy: 65.625%

Batch: 130, Accuracy: 68.75%

Batch: 140, Accuracy: 65.625%

Batch: 150, Accuracy: 68.75%

Batch: 160, Accuracy: 56.25%

